In [2]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch
import pandas as pd
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import HTTPError

import time

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [6]:
client =  Groq(api_key = os.environ['GROQ_API_KEY1'])
def llm(prompt, model = 'mixtral-8x7b-32768'):
    retries = 5
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model= 'llama3-8b-8192',
                messages=[{"role": "user", "content": prompt}]
            )
            json_response = response.choices[0].message.content
            return json_response
        except HTTPError as e:
            if e.response.status_code == 429:  # Rate limit error
                retry_after = float(e.response.json()['error']['message'].split('in ')[-1].split('s')[0])
                time.sleep(retry_after)
            else:
                raise
        except Exception as e:
            if i < retries - 1:
                time.sleep(2 ** i)  # Exponential backoff
            else:
                raise

In [9]:
results = []
for i in range(1,83):
    
    with open(f"../data/vietnamese_rag/llm_answer/llm_answer{i+82}.pkl", 'rb') as file:
        r = pickle.load(file)
    results.extend(r)
df = pd.DataFrame(results)

In [13]:
df.drop_duplicates(inplace = True)

In [14]:
df

,answer_llm,answer_orig,document,question,group
0,"Bé trai bị xe cán, nhưng cậu bé có sức chịu đự...",Cậu bé có sức chịu đựng dẻo dai và có thể bật ...,7451c351,Bé trai này đã có phản ứng gì sau khi bị xe cán?,General
1,Mẹ của cậu bé đã rất kinh hoàng khi xem đoạn v...,Cậu bé có sức chịu đựng dẻo dai và có thể bật ...,7451c351,Bố mẹ của cậu bé có phản ứng như thế nào khi x...,General
2,"Sau khi bị tai nạn, cậu bé được đưa đến Bệnh v...",Cậu bé có sức chịu đựng dẻo dai và có thể bật ...,7451c351,Cậu bé đã được kiểm tra sức khỏe ở đâu sau khi...,General
3,"Căn cứ vào thông tin từ Bệnh viện Nhi T.Ư, tìn...",Cậu bé có sức chịu đựng dẻo dai và có thể bật ...,7451c351,Các bác sĩ đưa ra kết luận gì về tình trạng sứ...,General
4,HLV Sundramoorthy trả lời phỏng vấn ngắn gọn v...,HLV Sundramoorthy của Singapore chỉ trả lời ph...,201f894a,HLV Sundramoorthy có lý do gì trả lời phỏng vấ...,General
...,...,...,...,...,...
1227,Chương trình kiểm tra an ninh toàn cầu do ICAO...,CAAT yêu cầu Không quân Hoàng gia Thái Lan tăn...,38f82fa7,Tên chương trình kiểm tra an ninh toàn cầu do ...,General
1228,Chula Sukmanop không có thông tin trong ngữ cả...,CAAT yêu cầu Không quân Hoàng gia Thái Lan tăn...,38f82fa7,Chula Sukmanop là ai?,General
1229,"Huỳnh Lập đã chọn MV ""Em gái mưa"" của Hương Tr...",MV 'Em gái mưa' của Hương Tràm được sản xuất b...,1b470510,Hương Tràm có MV nào được sản xuất bởi Alien M...,General
1230,Công ty sản xuất MV 'Em gái mưa' là Công ty TN...,MV 'Em gái mưa' của Hương Tràm được sản xuất b...,1b470510,Công ty sản xuất MV 'Em gái mưa' là ai?,General
